# P2P Lending Analytics

In [1]:
import numpy as np
import pandas as pd

In [45]:
cash_df = pd.DataFrame(columns=['platform_account_name','nonib_cash','ib_cash'])
concat_columns = -21 # no of columns required in the clean concatinated data frame

def platform_data_header(): # function for data enrichment common to all platforms e.g. platform name and sector
    df['platform_account_name'] = platform_account_name
    df['platform_account_sector'] = platform_account_sector

def platform_data_footer(): # function for data enrichment common to all platforms e.g. platform weighted values
    cash_df.loc[len(cash_df)] = { 'platform_account_name': platform_account_name, 'nonib_cash': nonib_cash, 'ib_cash': ib_cash } # set ib and nonib cash values
    platform_weight = df['loan_part_amt']/( df['loan_part_amt'].sum() + ib_cash )
    portfolio_weight_exc_negttm = df['loan_part_amt']/df.loc[df['loan_part_ttm_days'] >= 0, 'loan_part_amt'].sum() # weighting exc. negative time to maturity (principal default)
    portfolio_weight_exc_negterm = df['loan_part_amt']/df.loc[df['loan_part_term_months'] >= 0, 'loan_part_amt'].sum() # weighting excl. negative term
    df['platform_weighted_net_aer'] = df['loan_part_net_aer']*platform_weight
    df['platform_weighted_blended_ltv'] = df['loan_part_blended_ltv']*platform_weight
    df.loc[df['loan_part_term_months'] >= 0, 'platform_weighted_term_months'] = df['loan_part_term_months'] * portfolio_weight_exc_negterm # weighted term if not negative
    df.loc[df['loan_part_ttm_days'] >= 0, 'platform_weighted_ttm_months'] = df['loan_part_ttm_days'] / 30.44 * portfolio_weight_exc_negttm # weighted ttm if not negative

def platform_data_print():
    print("Loan amt: {}".format(df['loan_part_amt'].sum()))
    print("Int bearing amt: {}".format(df['loan_part_amt'].sum()+ib_cash))
    print("Non-IB cash: {}".format(nonib_cash))
    print("Total amt: {}".format(df['loan_part_amt'].sum()+ib_cash+nonib_cash))

#### Loanpad

In [44]:
##########################
platform_account_name = 'Loanpad 60d Access'
platform_account_sector = 'Development'
gross_apr = 0.065 # need to set APR manually for access account
nonib_cash = 6.84 # need to set cash account that loanpad pays interest into. this is non-interest bearing until re-invested to access account
##########################
df = pd.read_csv('data/loanpad.csv')
platform_data_header() # function for data enrichment common to all platforms e.g. platform name and sector
df['loan_part_name'] = df['Loan ID'].astype('str')
df['loan_part_id'] = df['Loan ID']
df['loan_part_status'] = df['Loan status']
df['loan_part_amt'] = pd.to_numeric(df['Your investment'].str.replace('£', ''))
df['loan_part_term_months'] = 2 # 60 d access account
df['loan_part_starts'] = np.nan # not provided
df['loan_part_matures'] = pd.to_datetime(df['Maturity'], errors='coerce')
df['loan_part_original_maturity'] = pd.to_datetime(df['Original maturity date'], dayfirst=True, errors='coerce')
df['loan_part_ttm_days'] = 60 # 60 d access account
df['loan_part_blended_ltv'] = pd.to_numeric(df['Loan to value'].str.replace('%', '').str.replace('-', ''))/100
df['loan_part_gross_apr'] = gross_apr
df['loan_part_net_apr'] = df['loan_part_gross_apr'] # loanpad does not charge fees on apr / aer received
loanpad_platf_amt = df['loan_part_amt'].sum()
gbp_apr_per_day = df['loan_part_net_apr']/365*loanpad_platf_amt # loanpad pay APR daily
no_days_to_gbp_10 = pd.to_numeric(10 / gbp_apr_per_day) # loanpad automatically reinvest in increments of GBP 10
df['loan_part_int_per'] = (365 / no_days_to_gbp_10).astype('int')
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # loanpad does not charge fees on apr / aer received
df['loan_parts_in_loan'] = df['Loan Parts in Loan']
ib_cash_row = df.loc[df['loan_part_name'] == 'Cash awaiting investment'] # find ib cash value in df
ib_cash = ib_cash_row.loan_part_amt.values[0]
df.drop(ib_cash_row.index.values[0], inplace=True) # drop ib cash value in df
platform_data_footer() # function for data enrichment common to all platforms e.g. platform weighted values
loanpad_df = df.iloc[:,concat_columns:]
platform_data_print() # print data

Loan amt: 5311.783026
Int bearing amt: 5450.0
Non-IB cash: 6.84
Total amt: 5456.84


#### PropLend

In [47]:
##########################
platform_account_name = 'Proplend'
platform_account_sector = 'Commercial'
nonib_cash = 16.49 # need to set cash account that proplend pays interest into. this is non-interest bearing until re-invested to access account
##########################
ib_cash = 0.0 # PropLend has no interest bearing cash account
df = pd.read_csv('data/proplend.csv')
platform_data_header() # function for data enrichment common to all platforms e.g. platform name and sector
df['loan_part_name'] = df['Loan Name']
df['loan_part_id'] = df['Loan Name'].astype('str')
df['loan_part_status'] = df['Status']
df['loan_part_amt'] = pd.to_numeric(df['Amount'].str.replace(',', ''))
df['loan_part_term_months'] = pd.to_numeric(df['Duration'])
df['loan_part_starts'] = np.nan
df['loan_part_matures'] = pd.to_datetime(df['End Date'], dayfirst=True)
df['loan_part_original_maturity'] = df['loan_part_matures'] # PropLend does not extend maturity dates
df['loan_part_ttm_days'] = (df['loan_part_matures'] - pd.to_datetime('today')).dt.days
df['loan_part_blended_ltv'] = pd.to_numeric(df['LTV'].str.replace('%', ''))/100
df['loan_part_gross_apr'] = pd.to_numeric(df['Interest Rate'])/100
df['loan_part_net_apr'] = df['loan_part_gross_apr']*.9 # proplend charge 10% fee on gross apr received from borrower
df['loan_part_int_per'] = 12 # proplend roll up interest and pay apr monthly
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # proplend charge a fee on APR
df['loan_parts_in_loan'] = 1 # proplend do not tranche loans so its always 1 loan part to 1 loan
platform_data_footer() # function for data enrichment common to all platforms e.g. platform weighted values
proplend_df = df.iloc[:,concat_columns:]
platform_data_print() # print data
df.iloc[0]

Loan amt: 3000.0
Int bearing amt: 3000.0
Non-IB cash: 16.49
Total amt: 3016.49


Loan Name                        Wolverhampton Office Block
Tranche                                                   A
Status                               Active - Good Standing
Loan Parts                                                1
Interest Rate                                           5.9
Duration                                                 24
End Date                                         08-07-2024
Accrued Interest                                       2.79
Amount                                             1,000.00
Interest Still Due                                    39.33
LTV                                                  50.00%
Notes                                                   NaN
platform_account_name                              Proplend
platform_account_sector                          Commercial
loan_part_name                   Wolverhampton Office Block
loan_part_id                     Wolverhampton Office Block
loan_part_status                     Act

#### CrowdProperty

In [55]:
##########################
platform_account_name = 'CrowdProperty AutoInvest'
platform_account_sector = 'Development'
nonib_cash = 90.74 # need to set cash account that CrowProperty pays interest into. this is non-interest bearing until re-invested to auto-invest account
##########################
ib_cash = 0.0 # crowdproperty has no interest bearing cash account
df = pd.read_csv('data/crowdproperty.csv')
platform_data_header() # function for data enrichment common to all platforms e.g. platform name and sector
df['loan_part_name'] = df['Project']
df['loan_part_id'] = df['Bank Reference'].astype('str')
df['loan_part_status'] = df['Status']
df['loan_part_amt'] = pd.to_numeric(df['Outstanding Capital']).astype('float')
df['loan_part_matures'] = pd.to_datetime(df['Loan End'], format='%d/%m/%Y')
df['loan_part_original_maturity'] = pd.to_datetime(df['Original maturity date'], dayfirst=True, errors='coerce')
df['loan_part_starts'] = pd.to_datetime(df['Loan Start'], format='%d/%m/%Y')
df['loan_part_term_months'] = (df['loan_part_matures'] - df['loan_part_starts']).dt.days / 30.437
df['loan_part_ttm_days'] = (df['loan_part_matures'] - pd.to_datetime('today')).dt.days
df['loan_part_blended_ltv'] = pd.to_numeric(df['Blended LTV'].str.replace('%', ''))/100
df['loan_part_gross_apr'] = pd.to_numeric(df['Gross APR'].str.replace('%', ''))/100
df['loan_part_net_apr'] = pd.to_numeric(df['Net APR'].str.replace('%', ''))/100
df['loan_part_int_per'] = 1 # crowdproperty roll up interest and pay apr annually
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # crowdproperty charge a spread on APR
df['loan_parts_in_loan'] = df['Loan Parts in Loan']
loan_parts_repaid = df.loc[df['Status'] == 'Paid Back'].index.values
loan_parts_repaid = df.loc[df['Status'] == 'Paid Back in Full'].index.values
df.drop(loan_parts_repaid, inplace=True) # drop repaid loans from dataframe
platform_data_footer() # function for data enrichment common to all platforms e.g. platform weighted values
crowdproperty_df = df.iloc[:,concat_columns:]
platform_data_print() # print data

Loan amt: 1992.26
Int bearing amt: 1992.26
Non-IB cash: 90.74
Total amt: 2083.0


#### Invest & Fund

In [60]:
##########################
platform_account_name = 'Invest & Fund'
platform_account_sector = 'Development'
nonib_cash = 22.43 # need to set cash account that Invest & Fund pays interest into. this is non-interest bearing until re-invested to auto-invest account
##########################
ib_cash = 0.0 # i&f has no interest bearing cash account
df = pd.read_csv('data/invest_fund.csv')
platform_data_header() # function for data enrichment common to all platforms e.g. platform name and sector
df['loan_part_name'] = df['Company Name']
df['loan_part_id'] = df['Loan Ref'].astype('str')
df['loan_part_status'] = 'Active'
df['loan_part_amt'] = pd.to_numeric(df['Outstanding Balance']).astype('float')
df['loan_part_term_months'] = pd.to_numeric(df['Term Remaining'].str.split('/ ').str[1].str.replace(' days','')) / 30.437
df['loan_part_starts'] = np.nan
df['loan_part_matures'] = pd.to_datetime(df['Maturity Date'], format='%d-%b-%y')
df['loan_part_original_maturity'] = pd.to_datetime(df['Original maturity date'], dayfirst=True, errors='coerce')
df['loan_part_ttm_days'] = (df['loan_part_matures'] - pd.to_datetime('today')).dt.days
df['loan_part_blended_ltv'] = pd.to_numeric(df['Blended LTV'])/100
df['loan_part_gross_apr'] = np.nan # i&f publish apr as well as aer but don't provide apr in their file
df['loan_part_net_apr'] =  df['loan_part_gross_apr']
df['loan_part_int_per'] = 12 # i&f pay aer with monthly compounding, but the rate in their file is already aer
df['loan_part_gross_aer'] = pd.to_numeric(df['Interest Rate'].str.replace('%', ''))/100 # i&f only provide aer in their file
df['loan_part_net_aer'] = df['loan_part_gross_aer']-0.0075 # i&f charge 0.75% spread on AER
df['loan_parts_in_loan'] = df['Loan Parts in Loan']
platform_data_footer() # function for data enrichment common to all platforms e.g. platform weighted values
invest_fund_df = df.iloc[:,concat_columns:]
platform_data_print() # print data

Loan amt: 2640.95
Int bearing amt: 2640.95
Non-IB cash: 22.43
Total amt: 2663.3799999999997


#### Assetz Capital

In [62]:
##########################
platform_account_name = 'Assetz Capital 90d Access'
platform_account_sector = 'Commercial'
gross_apr = 0.04 # need to set APR manually for access account. This is the cap, actual could be lower month-to-month.
nonib_cash = 0 # need to set cash account that Assetz Capital pays interest and replayments  through the winddown. this is non-interest bearing and needs to be withdrawn.
ib_cash = 0.0 # assetzcapital has no interest bearing cash account
##########################
df = pd.read_csv('data/assetz_capital.csv')
platform_data_header() # function for data enrichment common to all platforms e.g. platform name and sector
df['loan_part_name'] = df['name']
df['loan_part_id'] = df['id'].astype('str')
df['loan_part_status'] = df['risk_category']
df['loan_part_amt'] = pd.to_numeric(df['your_total_holding']).astype('float')
df['loan_part_term_months'] = df['loan_term'].astype(int)
df['loan_part_starts'] = np.nan # assetzcapital export does not provide loan start / end dates
df['loan_part_matures'] = np.nan # assetzcapital export does not provide loan start / end dates
df['loan_part_original_maturity'] = np.nan # assetzcapital export does not provide loan start / end dates
df['loan_part_ttm_days'] = round(df['term_remaining']*33.44).astype(int)
df['loan_part_blended_ltv'] = pd.to_numeric(df['ltv'])/100
df['loan_part_gross_apr'] = pd.to_numeric(gross_apr)
df['loan_part_net_apr'] =  df['loan_part_gross_apr'] # assetzcapital cap APR
df['loan_part_int_per'] = 12 # assetzcapital pay APR monthly
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # assetzcapital pay a fixed/capped apr, no spread on the published apr rate
df['loan_parts_in_loan'] = 1 # assetzcapital generally do not tranche their loans, so its mostly 1 loan part to 1 loan
platform_data_footer() # function for data enrichment common to all platforms e.g. platform weighted values
assetz_capital_df = df.iloc[:,concat_columns:]

#### Kuflink

In [64]:
##########################
platform_account_name = 'Kuflink 24m Term 28133'
platform_account_sector = 'Development'
nonib_cash = 0.0 # need to set cash account, but will typrically be 0 as term account rolls up interest payments until expiry
ib_cash = 7.45 # if there is any "Unallocated Amount" in the mandate, it's still interest bearing
ltv = 0.6401 # need to set ltv of the pooled account https://invest.kuflink.co.uk/product/pool
gross_apr = 0.08 # need to set APR manually for term account. This is the cap, actual could be lower month-to-month.
term_start = pd.to_datetime('01/08/23', format='%d/%m/%y')
term_finish = pd.to_datetime('01/08/25', format='%d/%m/%y')
term_months = 24
##########################
df = pd.read_csv('data/kuflink.csv')
platform_data_header() # function for data enrichment common to all platforms e.g. platform name and sector
df['loan_part_name'] = df['Loan Name']
df['loan_part_id'] = df['Loan Id'].astype('str')
df['loan_part_status'] = df['Loan Status']
df['loan_part_amt'] = pd.to_numeric(df['Monetary Amount'].str.replace('£', '')).astype('float')
df['loan_part_matures'] = pd.to_datetime(df['Loan End Date'], dayfirst=True)
df['loan_part_original_maturity'] = pd.to_datetime(df['Original maturity date'], dayfirst=True, errors='coerce')
df['loan_part_starts'] = pd.to_datetime(df['Loan Advance Date'], dayfirst=True)
df['loan_part_term_months'] = term_months
df['loan_part_ttm_days'] = (term_finish - pd.to_datetime('today')).days
df['loan_part_blended_ltv'] = ltv
df['loan_part_gross_apr'] = gross_apr
df['loan_part_net_apr'] =  df['loan_part_gross_apr'] # kuflink term caps APR
df['loan_part_int_per'] = 1 # kuflink term pays interest annually
df['loan_part_gross_aer'] = (1+df['loan_part_net_apr']/df['loan_part_int_per'])**df['loan_part_int_per']-1
df['loan_part_net_aer'] = df['loan_part_gross_aer'] # kuflink pay a fixed/capped apr, no spread on the published apr rate
df['loan_parts_in_loan'] = df['Loan Parts in Loan']
platform_data_footer() # function for data enrichment common to all platforms e.g. platform weighted values
kuflink_df = df.iloc[:,concat_columns:]
platform_data_print() # print data
df.iloc[0]

Loan amt: 1042.6000000000001
Int bearing amt: 1050.0500000000002
Non-IB cash: 0.0
Total amt: 1050.0500000000002


Loan Name                          Safety Bay - ME1- Plot 6
Loan Id                                                 293
Deal Id                                                 251
Loan Advance Date                                 18/1/2019
Loan End Date                                     30/4/2024
Fraction of Investment Split     104255*44002966/5697123137
Loan Split Amount Percentage                          0.77%
Loan Status                           Pending Status Update
Loan Amount                                     £440,029.66
Monetary Amount                                       £8.05
Loan Parts in Loan                                        2
Unique loan name                                 Safety Bay
Original maturity date                           30/11/2023
Maturity date extended                                  152
platform_account_name                Kuflink 24m Term 28133
platform_account_sector                         Development
loan_part_name                     Safet

#### Concat & enrich

In [65]:
loans_df = pd.concat([loanpad_df, proplend_df, crowdproperty_df, invest_fund_df, assetz_capital_df, kuflink_df], axis=0, ignore_index=True)
loans_df['loan_portion'] = 1/loans_df['loan_parts_in_loan']
portfolio_ib_cash = cash_df['ib_cash'].sum()
portfolio_nonib_cash = cash_df['nonib_cash'].sum()
portfolio_weight = loans_df['loan_part_amt']/(loans_df['loan_part_amt'].sum() + portfolio_ib_cash) # weighting inc. interest bearing cash
portfolio_weight_inc_nonib_cash = loans_df['loan_part_amt']/(loans_df['loan_part_amt'].sum() + portfolio_ib_cash + portfolio_nonib_cash) # wighting inc. interest and non-interest bearing cash
portfolio_weight_exc_negttm = loans_df['loan_part_amt']/loans_df.loc[loans_df['loan_part_ttm_days'] >= 0, 'loan_part_amt'].sum() # weighting exc. negative time to maturity (principal default)
portfolio_weight_exc_negterm = loans_df['loan_part_amt']/loans_df.loc[loans_df['loan_part_term_months'] >= 0, 'loan_part_amt'].sum() # weighting exc. negative terms

loans_df['portfolio_weighted_net_aer'] = loans_df['loan_part_net_aer']*portfolio_weight
loans_df['portfolio_weighted_net_aer_inc_nonib_cash'] = loans_df['loan_part_net_aer']*portfolio_weight_inc_nonib_cash
loans_df['portfolio_weighted_blended_ltv'] = loans_df['loan_part_blended_ltv']*portfolio_weight
loans_df['portfolio_weighted_blended_ltv_inc_nonib_cash'] = loans_df['loan_part_blended_ltv']*portfolio_weight_inc_nonib_cash
loans_df['loan_part_maturity_extended_days'] = loans_df['loan_part_original_maturity'] - loans_df['loan_part_matures']
loans_df.loc[loans_df['loan_part_ttm_days'] >= 0, 'portfolio_weighted_ttm_months'] = loans_df['loan_part_ttm_days'] / 30.44 * portfolio_weight_exc_negttm # weighted ttm if not negative
loans_df.loc[loans_df['loan_part_term_months'] >= 0, 'portfolio_weighted_term_months'] = loans_df['loan_part_term_months'] * portfolio_weight_exc_negterm # weighted term if not negative

status_dict = { 'Extended (No ICF)': 'Active',
               'Active - Good Standing': 'Active',
               'Performing': 'Active', 
               'Pending Status Update': 'Active', 
               'Live': 'Active', 
               'Partially Repaid': 'Active', 
               'High': 'Active', 
               'Medium': 'Active', 
               'Medium High': 'Active', 
               'Medium Low': 'Active', 
               'Low': 'Active', 
               'Default': 'Interest Default', 
               'INTEREST IN ARREARS': 'Interest Default',
               'Default (No Suspension - No ICF)': 'Interest Default',
               'Loan not yet started': 'Pledged' }
for status in status_dict: # clean loanpart statuses
    loans_df['loan_part_status'].mask(loans_df['loan_part_status'] == status, status_dict[status], inplace=True)    

loans_df['loan_part_status'].mask(loans_df['loan_part_ttm_days'] <= 0, 'Short Default', inplace=True) # short default status if ttm is minus
loans_df['loan_part_status'].mask(loans_df['loan_part_ttm_days'] <= -90, 'Long Default', inplace=True) # lonng default status if ttm is less than -90 days

loans_df.to_csv('data/concatenated.csv', index=False)
cash_df.to_csv('data/cash.csv', index=False)